### Fine-tune GPT

In [1]:
import json
import openai
import time, random
from datasets import load_dataset
print ("OpenAI package version:%s"%(openai.__version__))

OpenAI package version:0.28.0


### Load data from Hugging Face Datasets

In [2]:
dataset = load_dataset(path="nguha/legalbench",name="nys_judicial_ethics")

In [3]:
dataset["train"].to_pandas()

,answer,index,question,year
0,No,0,If a judge reports an attorney for a substanti...,2010
1,No,1,Does a village justice need to disqualify them...,2010
2,No,2,Is a judge required to disclose their former e...,2010
3,No,3,Is it prohibited for a part-time judge who pra...,2010
4,Yes,4,Can a judge appear monthly on a local televisi...,2010
5,Yes,5,Is a judge required to report an attorney to t...,2010
6,Yes,6,Is a judge required to take appropriate action...,2010
7,Yes,7,Can a judge appoint a qualified former law cle...,2010


In [4]:
dataset['test'].to_pandas()

,answer,index,question,year
0,Yes,0,Is a judge required to disclose a former law c...,2010
1,Yes,1,Can the names of judges who are members of a b...,2010
2,Yes,2,Should the judge disqualify themselves from a ...,2010
3,Yes,3,Should a judge disqualify themselves from all ...,2010
4,No,4,Can a judge designate the traffic court clerk ...,2010
...,...,...,...,...
287,Yes,287,Is it permissible for the inquirer to share et...,2021
288,Yes,288,Can a part-time lawyer judge who was assigned ...,2021
289,No,289,Can a part-time lawyer judge ordinarily appear...,2021
290,No,290,Can a practicing part-time lawyer judge repres...,2021


### Since the nunber of training set is too small, we'll use test set

In [5]:
NUM_TRAIN      = 10
NUM_VALIDATION = 5

In [6]:
base_text = """
Imagine your are the New York State Unified Court System Advisory Committee on Judicial Ethics. You've received the following question(s). Answer them as either "Yes" or "No".
"""

In [7]:
train_jsonl_path = '../data/legalbench_nys_judicial_ethics_train.json1'
validation_jsonl_path  = '../data/legalbench_nys_judicial_ethics_validation.json1'
with open(train_jsonl_path, 'w') as f:
    for i in range(NUM_TRAIN):
        data = dataset['test'][i]
        line = {"messages": [{"role": "system", "content": base_text}, 
                    {"role": "user", "content": "Question: " + data['question']},
                    {"role": "assistant", "content": "Answer: " + data['answer']}]}
        f.write(json.dumps(line) + '\n')

with open(validation_jsonl_path, 'w') as f:
    for i in range(NUM_TRAIN,NUM_TRAIN+NUM_VALIDATION):
        data = dataset['test'][i]
        line = {"messages": [{"role": "system", "content": base_text}, 
                    {"role": "user", "content": "Question: " + data['question']},
                    {"role": "assistant", "content": "Answer: " + data['answer']}]}
        f.write(json.dumps(line) + '\n')
print ("[%s] saved."%(train_jsonl_path))
print ("[%s] saved."%(validation_jsonl_path))

[../data/legalbench_nys_judicial_ethics_train.json1] saved.
[../data/legalbench_nys_judicial_ethics_validation.json1] saved.


### Locate the key

In [8]:
key_path = '../key/rilab_key.txt'
print ('key_path:[%s]'%(key_path))

key_path:[../key/rilab_key.txt]


In [9]:
with open(key_path, 'r') as f: OPENAI_API_KEY = f.read()
openai.api_key = OPENAI_API_KEY

### Delete existing files (optional)

In [10]:
past_file_lists = openai.File.list()
for past_file in past_file_lists['data']:
    print (past_file)

{
  "object": "file",
  "id": "file-PhLUhZJ5l2VsqnLzXY0RsCpE",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 5106,
  "created_at": 1693543712,
  "status": "uploaded",
  "status_details": null
}
{
  "object": "file",
  "id": "file-WuxpCaSYmkq171yJVWGAGmbR",
  "purpose": "fine-tune-results",
  "filename": "step_metrics.csv",
  "bytes": 326,
  "created_at": 1693545518,
  "status": "uploaded",
  "status_details": null
}
{
  "object": "file",
  "id": "file-l5WBArbeZ1DKW79pm2USfrCn",
  "purpose": "fine-tune-results",
  "filename": "step_metrics.csv",
  "bytes": 327,
  "created_at": 1693547402,
  "status": "uploaded",
  "status_details": null
}
{
  "object": "file",
  "id": "file-a3z1jd3Kt5giIVLsxPubxGUH",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 4841,
  "created_at": 1693548220,
  "status": "processed",
  "status_details": null
}
{
  "object": "file",
  "id": "file-LTE83jTd01XEUQ6aiL1iLeuN",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 2415,
  "cr

In [11]:
past_file_lists = openai.File.list()
for past_file in past_file_lists['data']:
    idx = past_file['id']
    if past_file['status'] == 'processed':
        print (past_file)
        openai.File.delete(idx)

{
  "object": "file",
  "id": "file-a3z1jd3Kt5giIVLsxPubxGUH",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 4841,
  "created_at": 1693548220,
  "status": "processed",
  "status_details": null
}
{
  "object": "file",
  "id": "file-LTE83jTd01XEUQ6aiL1iLeuN",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 2415,
  "created_at": 1693548234,
  "status": "processed",
  "status_details": null
}
{
  "object": "file",
  "id": "file-KQDYaX1BlnGQZHhk94Qjy35K",
  "purpose": "fine-tune-results",
  "filename": "step_metrics.csv",
  "bytes": 328,
  "created_at": 1693549011,
  "status": "processed",
  "status_details": null
}


In [12]:
# Print after deleting files
past_file_lists = openai.File.list()
for past_file in past_file_lists['data']:
    print (past_file)

{
  "object": "file",
  "id": "file-l5WBArbeZ1DKW79pm2USfrCn",
  "purpose": "fine-tune-results",
  "filename": "step_metrics.csv",
  "bytes": 327,
  "created_at": 1693547402,
  "status": "uploaded",
  "status_details": null
}
{
  "object": "file",
  "id": "file-WuxpCaSYmkq171yJVWGAGmbR",
  "purpose": "fine-tune-results",
  "filename": "step_metrics.csv",
  "bytes": 326,
  "created_at": 1693545518,
  "status": "uploaded",
  "status_details": null
}
{
  "object": "file",
  "id": "file-PhLUhZJ5l2VsqnLzXY0RsCpE",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 5106,
  "created_at": 1693543712,
  "status": "uploaded",
  "status_details": null
}


### Upload the dataset

In [13]:
file_train_data = openai.File.create(
  file=open(train_jsonl_path, "rb"),
  purpose='fine-tune'
)
file_train_data_id = file_train_data.id

In [14]:
file_test_data = openai.File.create(
  file=open(validation_jsonl_path, "rb"),
  purpose='fine-tune'
)
file_test_data_id = file_test_data.id

In [15]:
# Print after deleting files
past_file_lists = openai.File.list()
for past_file in past_file_lists['data']:
    print (past_file)

{
  "object": "file",
  "id": "file-WuxpCaSYmkq171yJVWGAGmbR",
  "purpose": "fine-tune-results",
  "filename": "step_metrics.csv",
  "bytes": 326,
  "created_at": 1693545518,
  "status": "uploaded",
  "status_details": null
}
{
  "object": "file",
  "id": "file-l5WBArbeZ1DKW79pm2USfrCn",
  "purpose": "fine-tune-results",
  "filename": "step_metrics.csv",
  "bytes": 327,
  "created_at": 1693547402,
  "status": "uploaded",
  "status_details": null
}
{
  "object": "file",
  "id": "file-t4PDvppb0D5qHrU4tHrfumh1",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 2415,
  "created_at": 1693554217,
  "status": "uploaded",
  "status_details": null
}
{
  "object": "file",
  "id": "file-rtZF3Ru65feoLs3eutQJsJ0H",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 4841,
  "created_at": 1693554213,
  "status": "uploaded",
  "status_details": null
}
{
  "object": "file",
  "id": "file-PhLUhZJ5l2VsqnLzXY0RsCpE",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 5106,
  "cre

### Wait for your data to be processed

In [16]:
while True:
    files = openai.File.list()
    completed = True
    for f_idx,file in enumerate(files['data']):
        print(f_idx,file['id'], file['status'])
        if file['id'] == file_train_data_id or file['id'] == file_test_data_id:
            processed = (file['status'] == 'processed')
            completed = completed and processed
    if completed:
        break
    time.sleep(seconds=10)
print ("Ready.")

0 file-PhLUhZJ5l2VsqnLzXY0RsCpE uploaded
1 file-WuxpCaSYmkq171yJVWGAGmbR uploaded
2 file-l5WBArbeZ1DKW79pm2USfrCn uploaded
3 file-rtZF3Ru65feoLs3eutQJsJ0H processed
4 file-t4PDvppb0D5qHrU4tHrfumh1 processed
Ready.


### Cancel running models (optional)

In [17]:
# List 10 fine-tuning jobs
jobs = openai.FineTuningJob.list(limit=10)
jobs = jobs['data']
for job in jobs:
    print(job['id'], job['status'])
    job_id = job['id']
    completed = job['status'] != 'running'
    if not completed:
        # Cancel a job
        openai.FineTuningJob.cancel(job_id)

ftjob-o3N7MRwfHtLXgvyheaO7pEHr succeeded
ftjob-FyMhJGWK7LaGlu94NuAadINS cancelled
ftjob-QcdcVVyLjBabDRhESarNTCpm succeeded
ftjob-fhipvS2c2q7G3t4s7CLQGgDH succeeded
ftjob-gNALz0Stw06p7e0NeI0isSAO succeeded
ftjob-DAZc9L3aEKg1mNY2zFmpgdhv succeeded


### Start fine-tuning

In [18]:
current_job = openai.FineTuningJob.create(
    training_file=file_train_data_id, model="gpt-3.5-turbo",
    validation_file=file_test_data_id, hyperparameters={"n_epochs":1, })
print ("Ready.")

Ready.


In [19]:
# List 10 fine-tuning jobs
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x127f64680> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1693554274,
      "finished_at": null,
      "fine_tuned_model": null,
      "organization_id": "org-bT1bA6ExTcdQphsyv85F0j6z",
      "result_files": [],
      "status": "running",
      "validation_file": "file-t4PDvppb0D5qHrU4tHrfumh1",
      "training_file": "file-rtZF3Ru65feoLs3eutQJsJ0H",
      "hyperparameters": {
        "n_epochs": 1
      },
      "trained_tokens": null
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-o3N7MRwfHtLXgvyheaO7pEHr",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1693548543,
      "finished_at": 1693549009,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:korea-university::7trkYzgO",
      "organization_id": "org-bT1bA6ExTcdQphsyv85F0j6z",
      "result_files": [
        "file-KQDYaX1BlnGQZHhk94Qjy35K"
    

### Retrieve information

In [20]:
current_job_id = current_job['id']
print  ("current_job_id:[%s]"%(current_job_id))

current_job_id:[ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ]


In [21]:
# Retrieve the state of a fine-tune
openai.FineTuningJob.retrieve(current_job_id)

<FineTuningJob fine_tuning.job id=ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ at 0x127f66e00> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693554274,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-bT1bA6ExTcdQphsyv85F0j6z",
  "result_files": [],
  "status": "running",
  "validation_file": "file-t4PDvppb0D5qHrU4tHrfumh1",
  "training_file": "file-rtZF3Ru65feoLs3eutQJsJ0H",
  "hyperparameters": {
    "n_epochs": 1
  },
  "trained_tokens": null
}

### Wait for fine-tuning

In [22]:
while True:
    job = openai.FineTuningJob.retrieve(current_job_id)
    completed = job['status'] != 'running'
    # List up to 10 events from a fine-tuning job
    events = openai.FineTuningJob.list_events(id=current_job_id, limit=2)
    for event in events['data']:
        print(event['message'])
    if completed:
        break
    time.sleep(60)
events = openai.FineTuningJob.list_events(id=current_job_id, limit=10)
for event in events['data']:
    print("=====================================")
    print(event['message'])
    print(event['data'])
print ("Ready.")

Fine tuning job started
Created fine-tune: ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ
Fine tuning job started
Created fine-tune: ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ
Fine tuning job started
Created fine-tune: ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ
Fine tuning job started
Created fine-tune: ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ
Fine tuning job started
Created fine-tune: ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ
Fine tuning job started
Created fine-tune: ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ
Fine tuning job started
Created fine-tune: ftjob-TxmbAGAFPoNgDwGdNr9kM7FJ
Step 2/10: training loss=3.19, validation loss=1.57
Step 1/10: training loss=2.89, validation loss=3.36
Fine-tuning job successfully completed
New fine-tuned model created: ft:gpt-3.5-turbo-0613:korea-university::7ttF0Dx2
Fine-tuning job successfully completed
None
New fine-tuned model created: ft:gpt-3.5-turbo-0613:korea-university::7ttF0Dx2
None
Step 10/10: training loss=0.01, validation loss=0.07
{
  "step": 10,
  "train_loss": 0.0061782835982739925,
  "valid_loss": 0.069464

### Inference

In [23]:
model_id = openai.FineTuningJob.retrieve(current_job_id)['fine_tuned_model']

In [24]:
start_idx = NUM_TRAIN + NUM_VALIDATION
end_idx = len(dataset['test'])
sample_idx = random.randint(start_idx, end_idx)
sample = dataset['test'][sample_idx]
print(sample)

{'answer': 'Yes', 'index': '87', 'question': 'Is a judicial candidate or their campaign committee allowed to post photographs on social media of the candidate together with sitting judges at a public or professional event, if the photograph was published with the consent of the judges and there is no verbiage that indicates an endorsement?', 'year': '2022'}


In [25]:
base_text

'\nImagine your are the New York State Unified Court System Advisory Committee on Judicial Ethics. You\'ve received the following question(s). Answer them as either "Yes" or "No".\n'

In [26]:
messages = [
    {"role": "system", "content": base_text},
    {"role": "user", "content": "Question: " + sample['question']}
]
print (messages)

[{'role': 'system', 'content': '\nImagine your are the New York State Unified Court System Advisory Committee on Judicial Ethics. You\'ve received the following question(s). Answer them as either "Yes" or "No".\n'}, {'role': 'user', 'content': 'Question: Is a judicial candidate or their campaign committee allowed to post photographs on social media of the candidate together with sitting judges at a public or professional event, if the photograph was published with the consent of the judges and there is no verbiage that indicates an endorsement?'}]


In [27]:
response = completion = openai.ChatCompletion.create(
  model=model_id,
  messages=messages
)

In [28]:
answer = response['choices'][0]['message']['content']
print('Question: ' + sample['question'])
print('Answer: ' + answer)
print('Ground Truth: ' + sample['answer'])

Question: Is a judicial candidate or their campaign committee allowed to post photographs on social media of the candidate together with sitting judges at a public or professional event, if the photograph was published with the consent of the judges and there is no verbiage that indicates an endorsement?
Answer: Answer: Yes
Ground Truth: Yes
